In [12]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer


In [17]:
import numpy as np 
import pandas as pd 

#submission_1 = pd.read_csv("F:/kaggle_Lakshmi_worked/submission.csv")    #only tes data
#submission_2 = pd.read_csv("F:/one_more_blend.csv")                 #only tet data
submission_3 = pd.read_csv("superblend_1.csv")




In [18]:
imp_text = pd.read_csv('kaggle_proj_Lakshmi/kaggle_Lakshmi_working/3.important words/imptext.csv')
imp_text.shape

(312735, 1)

In [19]:

class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('kaggle_proj_Lakshmi/train.csv').fillna('No data')
test = pd.read_csv('kaggle_proj_Lakshmi/test.csv').fillna('No data')

#train.id = train.id.astype('str')
#train.comment_text = train.comment_text.astype('str')

#test.id = test.id.astype('str')
#test.comment_text = test.comment_text.astype('str')


In [20]:
submission_3['comment_text'] = test.comment_text
submission_3.shape


(153164, 8)

In [34]:
new = submission_3.copy()
new = new[['id', 'comment_text']]
new['toxic'] = [1 if x >=0.4 else 0 for x in submission_3.toxic]
new['severe_toxic'] = [1 if x >=0.4 else 0 for x in submission_3.severe_toxic]
new['obscene'] = [1 if x >=0.4 else 0 for x in submission_3.obscene]
new['threat'] = [1 if x >=0.4 else 0 for x in submission_3.threat]
new['insult'] = [1 if x >=0.4 else 0 for x in submission_3.insult]
new['identity_hate'] = [1 if x >=0.4 else 0 for x in submission_3.identity_hate]

In [30]:
#submission_3.to_csv('super1.csv', index=False)

In [35]:
stacked = pd.concat([train, new])
stacked.shape

(312735, 8)

In [42]:
train_text = train[['id', 'comment_text']]
test_text = test[['id', 'comment_text']]
combined_text = stacked[['id', 'comment_text']]

In [8]:
print train_text.shape
print test_text.shape
print train_text.iloc[train_text.shape[0]-1, ]
print test_text.iloc[0, ]
print combined_text.iloc[train_text.shape[0]-1,]
print combined_text.iloc[train_text.shape[0],]

(159571, 2)
(153164, 2)
id                                               fff46fc426af1f9a
comment_text    "\nAnd ... I really don't think you understand...
Name: 159570, dtype: object
id                                               00001cee341fdb12
comment_text    Yo bitch Ja Rule is more succesful then you'll...
Name: 0, dtype: object
id                                               fff46fc426af1f9a
comment_text    "\nAnd ... I really don't think you understand...
Name: 159570, dtype: object
id                                               00001cee341fdb12
comment_text    Yo bitch Ja Rule is more succesful then you'll...
Name: 0, dtype: object


In [40]:
missing = imp_text.loc[imp_text.comment.isna(), ]
missing_index = list(missing.index)


In [43]:
#dont use errors='ignore'
combined_text.comment_text = map(lambda x:unicode(x, 'utf-8'), combined_text.comment_text)
#combined_text = map(lambda x:unicode(x, 'utf-8'), combined_text)

In [ ]:
#extracting the stopwords from nltk library
sw = stopwords.words('english')
# displaying the stopwords
np.array(sw)

In [45]:
def stopwords(text):
    '''a function for removing the stopword'''
# removing the stop words and lowercasing the selected words
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    # joining the list of words with space separator
    return " ".join(text)

In [46]:
combined_text['comment_text'] = combined_text['comment_text'].apply(stopwords)
combined_text.head(10)
#combined_text = map(stopwords, combined_text)
#combined_text

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,id,comment_text
0,0000997932d777bf,explanation edits made username hardcore metal...
1,000103f0d9cfb60f,d'aww! matches background colour i'm seemingly...
2,000113f07ec002fd,"hey man, i'm really trying edit war. guy const..."
3,0001b41b1c6bb37e,""" can't make real suggestions improvement - wo..."
4,0001d958c54c6e35,"you, sir, hero. chance remember page that's on?"
5,00025465d4725e87,""" congratulations well, use tools well. · talk """
6,0002bcb3da6cb337,cocksucker piss around work
7,00031b1e95af7921,vandalism matt shirvington article reverted. p...
8,00037261f536c51d,"sorry word 'nonsense' offensive you. anyway, i..."
9,00040093b2687caa,alignment subject contrary dulithgow


In [48]:
PunctuationToRemove = [".", ",", ":", ";", "!" ,"?", "&", "\"", "\'", "~",  "=", "@", "$", "[", "]", "(", ")", "/", "{", "}", "|", ">","<",  "\\", "+", "%", "#"]

def remove_punctuation(text):
    '''a function for removing punctuation'''
    for char in PunctuationToRemove:
        text = text.replace(char,"")
    return text
combined_text['comment_text'] = combined_text['comment_text'].apply(remove_punctuation)
combined_text.head(10)
#combined_text = map(remove_punctuation, combined_text)
#combined_text

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,comment_text
0,0000997932d777bf,explanation edits made username hardcore metal...
1,000103f0d9cfb60f,daww matches background colour im seemingly st...
2,000113f07ec002fd,hey man im really trying edit war guy constant...
3,0001b41b1c6bb37e,cant make real suggestions improvement - wond...
4,0001d958c54c6e35,you sir hero chance remember page thats on
5,00025465d4725e87,congratulations well use tools well · talk
6,0002bcb3da6cb337,cocksucker piss around work
7,00031b1e95af7921,vandalism matt shirvington article reverted pl...
8,00037261f536c51d,sorry word nonsense offensive you anyway im in...
9,00040093b2687caa,alignment subject contrary dulithgow


In [49]:
missing_values = list(combined_text.iloc[missing_index, 1])
#missing_values

In [50]:
imp_text1 = imp_text.copy()
imp_text1.iloc[missing_index, 0] = missing_values

In [55]:
#imp_text1

In [57]:
all_text = imp_text1.comment

In [58]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    encoding='utf-8',
    lowercase=True,
    #min_df=0.00001,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    #use_idf=1, smooth_idf=1,
    max_features=None)

word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(stacked.comment_text)
test_word_features = word_vectorizer.transform(test_text.comment_text)

msg = "There are {} tokens in Comment_text if we use word"
print(msg.format(len(word_vectorizer.get_feature_names())))

There are 25265 tokens in Comment_text if we use word


In [63]:
#train_features = hstack([train_char_features, train_word_features])
#test_features = hstack([test_char_features, test_word_features])
#X_train, X_valid, y_train, y_valid = train_test_split(train_features, train_target, test_size=0.3, random_state=42)


scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = stacked[class_name]
    classifier = LogisticRegression(solver='sag')

    cv_score = np.mean(cross_val_score(classifier, train_word_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_word_features, train_target)
    submission[class_name] = classifier.predict_proba(test_word_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

submission.to_csv('mysubmission3.csv', index=False)

CV score for class toxic is 0.986669734525
CV score for class severe_toxic is 0.991601882637
CV score for class obscene is 0.993873254319
CV score for class threat is 0.988791573613
CV score for class insult is 0.98646178656
CV score for class identity_hate is 0.988434403464
Total CV score is 0.989305439186


In [61]:
#train_features = hstack([train_char_features, train_word_features])
#test_features = hstack([test_char_features, test_word_features])

scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = stacked[class_name]
    X_train, X_valid, y_train, y_valid = train_test_split(train_word_features, train_target, test_size=0.3, random_state=42)


    classifier = LogisticRegression(solver='sag')

    cv_score = np.mean(cross_val_score(classifier, X_valid, y_valid, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(X_train, y_train)
    #submission[class_name] = classifier.predict_proba(test_word_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

#submission.to_csv('F:\impwordssub.csv', index=False)

CV score for class toxic is 0.981240128886
CV score for class severe_toxic is 0.987429907895
CV score for class obscene is 0.990806882068
CV score for class threat is 0.983957897517
CV score for class insult is 0.981914569155
CV score for class identity_hate is 0.981067329785
Total CV score is 0.984402785884


In [62]:
#train_features = hstack([train_char_features, train_word_features])
#test_features = hstack([test_char_features, test_word_features])

scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = stacked[class_name]
    X_train, X_valid, y_train, y_valid = train_test_split(train_word_features, train_target, test_size=0.3, random_state=42)


    classifier = MultinomialNB()

    cv_score = np.mean(cross_val_score(classifier, X_valid, y_valid, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(X_train, y_train)
    #submission[class_name] = classifier.predict_proba(test_word_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

#submission.to_csv('F:\impwordssub.csv', index=False)

CV score for class toxic is 0.946554920876
CV score for class severe_toxic is 0.936215919315
CV score for class obscene is 0.9355287361
CV score for class threat is 0.886947003746
CV score for class insult is 0.93722629103
CV score for class identity_hate is 0.914459874538
Total CV score is 0.926155457601
